1. Read json file
2. Set up the sampler through batchrunner
3. generate the input file
4. save it.

In [26]:
# Import relevant libraries
import os
from ModularCirc import BatchRunner
import numpy as np
import pandas as pd
from ModularCirc.Models.NaghaviModel import NaghaviModel, NaghaviModelParameters, TEMPLATE_TIME_SETUP_DICT
from SALib.analyze.sobol import analyze
from SALib.sample.sobol import sample as sobol_sample
from SALib.analyze.sobol import analyze as sobol_analyze
import matplotlib.pyplot as plt
from autoemulate.compare import AutoEmulate
from SALib.analyze.sobol import analyze
import numpy as np


In [27]:
os.getcwd()

'/Users/lbokeria/projects/health_gc/cvd_net_project/comparative-gsa/sandbox'

In [28]:
# Define variables
param_path = "./parameters_01.json"

n_samples = 2048 # Powers of 2 (n+2)1024 32768

simulation_out_path = f'outputs/simulations/output_{n_samples}_samples'

os.makedirs(os.path.join(simulation_out_path, 
                         f'raw_simulations_{n_samples}_samples'), 
                         exist_ok=True)

In [29]:
# Set up the batch runner
br = BatchRunner('Sobol', 0) # why are we using 'Sobol' here? 
br.setup_sampler(param_path)
br.sample(n_samples)

In [30]:
br.samples

,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,...,ven.v_ref,la.v,la.delay,la.t_tr,la.tau,la.t_max,lv.delay,lv.t_tr,lv.tau,lv.t_max
0,324.140512,0.429410,970.557289,2.593650,5.759509,141.391649,5.535208,3.645771,0.302181,0.614511,...,2800,93,150,225,25,150,0,420,25,280
1,213.211374,0.168363,1134.582900,3.092750,11.457833,81.051107,6.337772,5.927350,0.476733,0.449234,...,2800,93,150,225,25,150,0,420,25,280
2,128.027561,0.373035,664.791603,1.807716,9.775524,110.934879,3.831676,4.690519,0.657067,0.244713,...,2800,93,150,225,25,150,0,420,25,280
3,287.120769,0.262023,1448.888220,3.761496,8.444797,167.239972,7.529157,2.905174,0.338195,0.522401,...,2800,93,150,225,25,150,0,420,25,280
4,261.317508,0.322630,1342.111306,4.190009,7.184780,154.969280,3.238999,4.237066,0.583734,0.372392,...,2800,93,150,225,25,150,0,420,25,280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2043,261.101611,0.261382,685.537472,3.721722,9.839757,78.661601,3.416618,4.155957,0.280508,0.271551,...,2800,93,150,225,25,150,0,420,25,280
2044,287.329562,0.323235,998.297711,3.291562,10.530699,133.110328,8.127906,3.052351,0.529915,0.619758,...,2800,93,150,225,25,150,0,420,25,280
2045,128.141604,0.283698,1396.881532,2.277480,7.231221,189.277109,4.324956,4.855710,0.245566,0.342032,...,2800,93,150,225,25,150,0,420,25,280
2046,213.119531,0.379612,741.421257,4.124279,5.566558,152.640929,6.506589,6.052502,0.394962,0.320269,...,2800,93,150,225,25,150,0,420,25,280


In [31]:
map_ = {
    'lv.t_tr' : ['lv.t_tr',],
    'la.t_tr' : ['la.t_tr',],
    'la.delay' : ['la.delay',],
    'lv.tau' : ['lv.tau',],
    'la.tau' : ['la.tau',],
    'lv.t_max' : ['lv.t_max',],
    'la.t_max' : ['la.t_max',],
}
br.map_sample_timings(
    ref_time=1000., # double check if 1000 or 1
    map=map_
    )

In [32]:
br.map_vessel_volume()

In [33]:
br.samples

,ao.r,ao.c,art.r,art.c,ven.r,ven.c,av.r,mv.r,la.E_pas,la.E_act,...,la.t_tr,la.tau,la.t_max,lv.delay,lv.t_tr,lv.tau,lv.t_max,ao.v,art.v,ven.v
0,324.140512,0.429410,970.557289,2.593650,5.759509,141.391649,5.535208,3.645771,0.302181,0.614511,...,103.265426,11.473936,68.843618,0,192.762129,11.473936,128.508086,11.183394,67.548099,3682.353190
1,213.211374,0.168363,1134.582900,3.092750,11.457833,81.051107,6.337772,5.927350,0.476733,0.449234,...,264.970053,29.441117,176.646702,0,494.610765,29.441117,329.740510,12.893425,236.845967,6206.977557
2,128.027561,0.373035,664.791603,1.807716,9.775524,110.934879,3.831676,4.690519,0.657067,0.244713,...,173.562758,19.284751,115.708506,0,323.983815,19.284751,215.989210,14.695447,71.213695,4370.200691
3,287.120769,0.262023,1448.888220,3.761496,8.444797,167.239972,7.529157,2.905174,0.338195,0.522401,...,132.781531,14.753503,88.521021,0,247.858858,14.753503,165.239239,8.718682,125.161948,5564.827928
4,261.317508,0.322630,1342.111306,4.190009,7.184780,154.969280,3.238999,4.237066,0.583734,0.372392,...,146.342358,16.260262,97.561572,0,273.172401,16.260262,182.114934,10.757573,139.708922,5167.193956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2043,261.101611,0.261382,685.537472,3.721722,9.839757,78.661601,3.416618,4.155957,0.280508,0.271551,...,217.053788,24.117088,144.702525,0,405.167071,24.117088,270.111381,12.615141,179.622480,3796.466346
2044,287.329562,0.323235,998.297711,3.291562,10.530699,133.110328,8.127906,3.052351,0.529915,0.619758,...,178.426114,19.825124,118.950743,0,333.062079,19.825124,222.041386,8.529734,86.859773,3512.597985
2045,128.141604,0.283698,1396.881532,2.277480,7.231221,189.277109,4.324956,4.855710,0.245566,0.342032,...,137.632455,15.292495,91.754970,0,256.913915,15.292495,171.275944,9.114066,73.166307,6080.715633
2046,213.119531,0.379612,741.421257,4.124279,5.566558,152.640929,6.506589,6.052502,0.394962,0.320269,...,98.426772,10.936308,65.617848,0,183.729974,10.936308,122.486650,10.688504,116.124934,4297.821799


In [34]:
# Save the samples to a CSV file

br.samples.to_csv(os.path.join(simulation_out_path,
                               f'input_samples_{n_samples}.csv'),
                                 index=False)

In [35]:
br.setup_model(model=NaghaviModel, po=NaghaviModelParameters,
                   time_setup=TEMPLATE_TIME_SETUP_DICT)

In [36]:
simulations = br.run_batch(
            n_jobs=10,
            output_path=os.path.join(simulation_out_path, f'raw_simulations_{n_samples}_samples') 
        )

100%|██████████| 2048/2048 [02:48<00:00, 12.16it/s]


In [37]:
import json

bool_indices = [index for index, value in enumerate(simulations) if isinstance(value, bool)]

with open(os.path.join(simulation_out_path,"bool_indices.json"), 'w') as f:
    json.dump(bool_indices, f)

print(bool_indices)  

[31, 438, 855, 872, 1211, 1459, 1489, 1608, 1840]


In [38]:
## Setting up the headers of the columns for my simulation summary similar to fay
##headers = [
   # 'v_ao', 'v_art', 'v_ven', 'v_la', 'v_lv', 'p_lv', 'q_av', 'p_ao', 'p_art',
   # 'q_ao', 'p_ven', 'q_art', 'p_la', 'q_ven', 'q_mv', 'T'
#]

summary_rows = []

for i_sim in range(len(simulations)): ## stuck with the length of simulations like fay
    sim_df = simulations[i_sim]
    
    # Skip if not a DataFrame this was needed because sometimes simulations can return None or other types. or false.
    if not isinstance(sim_df, pd.DataFrame):
        print(f"Simulation {i_sim} is not a DataFrame, skipping.")
        continue

    row = {} ##the 'dictiionary' to hold the summary for each simulation
    for h in sim_df.columns: # We had sim_df the whole time which means h can be used without the headers list!
        row[f"{h}_mean"] = sim_df[h].mean()
        row[f"{h}_max"] = sim_df[h].max() ## using the headers defined above to make the new csv file headers eg v_ao_mean, v_ao_max, etc.
    summary_rows.append(row) # this is a list that is being used to collect summary data for our simulations.
    print(f"Simulation {i_sim} done.") # simple way to track progress 

summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(os.path.join(simulation_out_path, "simulations_summary.csv"), index=False)  ## the location, nice and organised!
print("Saved as simulation_summary.csv") # not really needed but nice to show the file has been saved! :D

Simulation 0 done.
Simulation 1 done.
Simulation 2 done.
Simulation 3 done.
Simulation 4 done.
Simulation 5 done.
Simulation 6 done.
Simulation 7 done.
Simulation 8 done.
Simulation 9 done.
Simulation 10 done.
Simulation 11 done.
Simulation 12 done.
Simulation 13 done.
Simulation 14 done.
Simulation 15 done.
Simulation 16 done.
Simulation 17 done.
Simulation 18 done.
Simulation 19 done.
Simulation 20 done.
Simulation 21 done.
Simulation 22 done.
Simulation 23 done.
Simulation 24 done.
Simulation 25 done.
Simulation 26 done.
Simulation 27 done.
Simulation 28 done.
Simulation 29 done.
Simulation 30 done.
Simulation 31 is not a DataFrame, skipping.
Simulation 32 done.
Simulation 33 done.
Simulation 34 done.
Simulation 35 done.
Simulation 36 done.
Simulation 37 done.
Simulation 38 done.
Simulation 39 done.
Simulation 40 done.
Simulation 41 done.
Simulation 42 done.
Simulation 43 done.
Simulation 44 done.
Simulation 45 done.
Simulation 46 done.
Simulation 47 done.
Simulation 48 done.
Simula